In [ ]:
%load_ext aiida
%aiida

In [ ]:
from aiidalab_atmospec_workchain import OrcaWignerSpectrumWorkChain
from aiida.engine import WorkChain, calcfunction
from aiida.engine import submit, run, append_, ToContext, if_
from aiida.engine import run_get_node, run_get_pk

StructureData = DataFactory("core.structure")
Dict = DataFactory("core.dict")
TrajectoryData = DataFactory("core.array.trajectory")

In [ ]:
# https://github.com/aiidateam/aiida-core/blob/2c183fc4486e00f3348a1b66cdcd6d9fbfd563f0/.github/system_tests/workchains.py#L182

# General WorkChain for combining all inputs from a dynamic namespace 'ns'
# into a single List.
# Used to combine outputs from several subworkflows into one output
# It should be launched via run() instead of submit()
class CombineInputsToList(WorkChain):
    
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input_namespace("ns", dynamic=True)
        spec.output("output", valid_type=List)
        spec.outline(cls.combine)
        
    def combine(self):
        #input_list = [self.inputs.ns[k] for k in self.inputs.ns]
        input_list = [self.inputs.ns[k].get_dict() if isinstance(self.inputs.ns[k], Dict) else self.inputs.ns[k] for k in self.inputs.ns]
        self.out('output', List(list=input_list).store())
        
        
class CombineStructuresToTrajectoryData(WorkChain):
    
    @classmethod
    def define(cls, spec):
        super().define(spec)
        # TODO: Maybe allow other types other than StructureData?
        # Not sure what are the requirements for TrajectoryData
        spec.input_namespace("structures", dynamic=True, valid_type=StructureData)
        spec.output("trajectory", valid_type=TrajectoryData)
        spec.outline(cls.combine)
        
    def combine(self):
        structurelist = [self.inputs.structures[k] for k in self.inputs.structures]
        self.out('trajectory', TrajectoryData(structurelist=structurelist).store())

# Now test more than one conformer

In [ ]:
builder = AtmospecWorkChain.get_builder()
old_workchain = load_node(pk=226)
builder.structure = old_workchain.inputs.structure
for input in old_workchain.inputs:
    if input != 'structure':
        builder[input] = old_workchain.inputs[input]
        
# Patch the inputs to reduct comp cost
builder.nwigner = 2

params = builder.opt.orca.parameters.get_dict()
params['input_keywords'] = ['sto-3g', 'pbe', 'Opt', 'AnFreq']
builder.opt.orca.parameters = Dict(dict=params)

params = builder.exc.orca.parameters.get_dict()
params['input_keywords'] = ['sto-3g', 'pbe']
builder.exc.orca.parameters = Dict(dict=params)

# Not sure why this is not already included
builder.opt.orca.metadata.options.resources = {'tot_num_mpiprocs': 1}
builder.exc.orca.metadata.options.resources = {'tot_num_mpiprocs': 1}
builder.opt.clean_workdir = Bool(True)
builder.exc.clean_workdir = Bool(True)
builder

In [ ]:
output = run(builder)
output

In [ ]:
x = Int(1).store()
y = Int(2).store()

In [ ]:
struct = load_node(pk=1824)
l = [x, y, struct]
# This doesn't work
inputs = {str(i): val for i, val in enumerate(l)}
#run(CombineInputsToList, ns=inputs)

In [ ]:
inputs

In [ ]:
l = [struct, struct]
inputs = {str(i): val for i, val in enumerate(l)}
traj = run(CombineStructuresToTrajectoryData, structures=inputs)
traj

In [ ]:
len(traj['trajectory'].get_stepids())

In [ ]:
l = [List(list=[1, 2]), List(list=[2, 3])]
inputs = {str(i): val for i, val in enumerate(l)}
run(CombineInputsToList, ns=inputs)

In [ ]:
l = [Dict(dict={"1": 2}), Dict(dict={"1": 2})]
inputs = {str(i): val for i, val in enumerate(l)}
run(CombineInputsToList, ns=inputs)

In [ ]:
l[0].get_dict()

In [ ]:
class ConcatDictsToList(WorkChain):
    
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input_namespace("ns", dynamic=True)
        spec.output("output", valid_type=List)
        spec.outline(cls.combine)
        
    def combine(self):
        input_list = [self.inputs.ns[k].get_dict() for k in self.inputs.ns]
        self.out('output', List(list=input_list).store())

In [ ]:
run(ConcatDictsToList, ns=inputs)

In [ ]:
from aiida.engine import calcfunction
import numpy as np
Array = DataFactory('core.array')
TrajectoryData = DataFactory('array.trajectory')

@calcfunction
def structures_to_trajectory(arrays: Array = None, **structures) -> TrajectoryData:
    traj = TrajectoryData([structure for structure in structures.values()])
    if arrays is not None:
        for name in arrays.get_arraynames():
            traj.set_array(name, arrays.get_array(name))
    return traj

In [ ]:
struct1 = load_node(224)
struct2 = load_node(213)
structures = {
    '0': struct1,
    '1': struct1,
}
energies = [0.0, 1.0]
arrays = Array()
arrays.set_array('energies', np.array(energies))
arrays.set_array('boltzmann_weights', np.array([2., 3.]))
traj = structures_to_trajectory(**structures)

In [ ]:
traj

In [ ]:
XyData = DataFactory('core.array.xy')
import numpy as np

xy = XyData()

In [ ]:
xy.set_x(np.array([3,4]), "Geometry index", "-")
y1 = np.array([2,1])
y2 = np.array([3,4])
labels = ["Energy", "BW"]
units = ["kJ/mol", "-"]
xy.set_y([y1,y2], labels, units)

In [ ]:
xy.get_arraynames()

In [ ]:
xy.set_y(np.array([0,1,3]), "Boltzmann weights", "-")

In [ ]:
xy.get_y()

In [ ]:
Array = DataFactory('core.array')

In [ ]:
ar = Array()

In [ ]:
ar.set_array("name", np.array([1,2]))

In [ ]:
ar.get_array("name")

In [ ]:
from aiidalab_atmospec_workchain import structures_to_trajectory

In [ ]:
structures_to_trajectory(**structures)

In [ ]:
structures_to_trajectory?

In [ ]:
structures